<a href="https://colab.research.google.com/github/jckaren1124/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://ppa

In [2]:
# Load Amazon data into Spark DataFrame
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [14]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

#convert helpful votes and total_votes to integer data type
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
# create the list of struct fields
schema = [StructField("helpful_votes", IntegerType(), True), StructField("total_votes", IntegerType(), True),]
# Pass in our fields
final = StructType(fields=schema)
dataframe = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), schema=final, sep="\t", header=True)
dataframe.printSchema()

root
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)



In [15]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [16]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [24]:
# 1. Filter DataFrame with total_votes greater than or equal to 20
high_vote_count_df = vine_df.filter( (df["total_votes"]>20) | (df["total_votes"] == 20) )
high_vote_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

In [26]:
# 2. Filter DataFrame where helpful_votes divided by total_votes is greater than 50%
ratings_percentage_df = high_vote_count_df.withColumn("ratings_percentage", high_vote_count_df["helpful_votes"]/ high_vote_count_df["total_votes"])
high_ratings_percentage_df = ratings_percentage_df.filter("ratings_percentage > 0.5")
high_ratings_percentage_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|ratings_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|0.8947368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             0.875|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|0.8846153846153846|
|R2M6WNM2OZZHKS|          1|  

In [28]:
# 3. Filter DataFrame for reviews under Vine program (paid)
paid_reviews_df = high_ratings_percentage_df.filter("vine == 'Y' ")
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|ratings_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N| 0.918918918918919|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|0.8648648648648649|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|              0.88|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|          0.953125|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|               1.0|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|0.9655172413793104|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|0.9806201550387597|
|R13EGJI8C8YJ4Y|          5|  

In [29]:
# 4. Filter DataFrame for reviews not in Vine program (unpaid)
unpaid_reviews_df = high_ratings_percentage_df.filter("vine == 'N' ")
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|ratings_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|0.8947368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             0.875|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|0.8846153846153846|
|R2M6WNM2OZZHKS|          1|  

In [57]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review 
#(paid vs unpaid).
total_reviews = high_ratings_percentage_df.count()
total_reviews

28235

In [58]:
total_five_star_reviews_df = high_ratings_percentage_df.filter("star_rating == 5")
total_five_star_reviews = total_five_star_reviews_df.count()
total_five_star_reviews

15692

In [59]:
paid_five_star_reviews_df = paid_reviews_df.filter("star_rating ==5")
paid_five_star_reviews = paid_five_star_reviews_df.count()
paid_five_star_reviews

20

In [60]:
unpaid_five_star_reviews_df = unpaid_reviews_df.filter("star_rating ==5")
unpaid_five_star_reviews = unpaid_five_star_reviews_df.count()
unpaid_five_star_reviews

15672

In [48]:
paid_five_star_percentage = paid_five_star_reviews/total_five_star_reviews*100
paid_five_star_percentage

0.1274534794799898

In [56]:
unpaid_five_star_percentage = unpaid_five_star_reviews/total_five_star_reviews*100
unpaid_five_star_percentage

99.87254652052002